In [1]:
import os
import torch
import re
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tokenizers import Tokenizer

In [2]:
import yaml

path = "./configs/config.yaml"

with open(path, "r") as f:
    config = yaml.safe_load(f)

In [3]:
from src.data_utils import process_data

cleaned = process_data(config["data"]["data_dir"], config["data"]["raw_path"], config["data"]["processed_path"])

Загрузка/обработка данных...
--Данные загружены из кэша


In [4]:
from src.tokenizer import train_tokenizer

train_tokenizer(config["data"]["processed_path"], config["data"]["tokenizer_file_name"])

Токенайзер уже прошел тренировку


In [5]:
from src.data_utils import data_tokenization

tokenized = data_tokenization(config["data"]["data_dir"], config["data"]["tokenized_file_name"], cleaned)

Загружено 1596901 последовательностей


In [6]:
from src.data_utils import data_split

train_seqs, val_seqs, test_seqs = data_split(tokenized)

📊 Train: 1277520, Val: 159690, Test: 159691


In [7]:
from src.data_utils import TweeterDataset

train_dataset = TweeterDataset(train_seqs)
val_dataset = TweeterDataset(val_seqs)
test_dataset = TweeterDataset(test_seqs)

In [8]:
from src.data_utils import collate_fn

train_dataloader = DataLoader(
    train_dataset,
    batch_size=config["data"]["train_batch_size"],
    shuffle=True,
    collate_fn=collate_fn,
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=config["data"]["val_batch_size"],
    collate_fn=collate_fn,
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=config["data"]["test_batch_size"],
    collate_fn=collate_fn,
)

In [14]:
from src.lstm_train import training, validation, evaluate_rouge_on_val, generate
tokenizer = Tokenizer.from_file(config["data"]["tokenizer_file_name"])

num_epochs = config["training"]["epochs"]
for epoch in range(num_epochs):
    print(f"Эпоха {epoch+1}/{num_epochs}")
    
    training(train_dataloader)
    should_stop = validation(val_dataloader)
    if should_stop: break

    if (epoch + 1) % 5 == 0 or epoch == 0:
        evaluate_rouge_on_val(test_dataloader, tokenizer)
    
    if (epoch + 1) % 2 == 0 or epoch == 0:
        generate(tokenizer)

TypeError: '<=' not supported between instances of 'float' and 'str'

In [13]:
from transformers import pipeline
from src.eval_transformer_pipeline import transformer_generate
from src.eval_transformer_pipeline import rouge_calc_for_transformer

# Примеры генерации трансформера
generation_prompts = ["I am about", "What is going on with", "Don't you mind if", "Let us deal with", "Deep Learning is"]
generator = pipeline("text-generation", model="distilgpt2")

transformer_generate(generator, generation_prompts)
rouge_calc_for_transformer(generator)

Loading weights: 100%|██████████| 76/76 [00:00<00:00, 652.89it/s, Materializing param=transformer.wte.weight]            


Примеры генерации:
1) I am about to give you a big thank you.”
2) What is going on with my life, I really do have to be able to take care of the body, that's a
3) Don't you mind if you don't like the idea?      The end result is: So
4) Let us deal with it with the same problem as a school student.   But in a world where our world
5) Deep Learning is a tool to learn and develop complex programming languages.


Using the latest cached version of the module from /home/ubuntu/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--rouge/6e5315f72865c2eaa764c8361360bb938740b9c120a2cf3a7ad218aa0ce452ed (last modified on Tue Feb 17 14:28:43 2026) since it couldn't be found locally at evaluate-metric--rouge, or remotely on the Hugging Face Hub.


ModuleNotFoundError: No module named 'absl'